# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
# Import Dependencies.

from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

import logging
import os

import azureml.core
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

from sklearn.model_selection import train_test_split

import pandas as pd
from azureml.widgets import RunDetails

import joblib 

from azureml.core.model import Model

from azureml.pipeline.core import PipelineData, TrainingOutput

from azureml.pipeline.core import Pipeline

import pickle

from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

from azureml.core.webservice import Webservice, AciWebservice

import requests
import json

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview

There are some factors that affects Death Event. We use Kaggle's dataset Heart Failure Clinical Records which contains  information like age, sex, blood pressure, smoke, diabetes, ejection fraction, creatinine phosphokinase, serum_creatinine, serum_sodium, time and we have to predict their death_event.

Dataset was uploaded and register as dataset in the workspace


In [2]:
# Create a workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


quick-starts-ws-132429
aml-quickstarts-132429
southcentralus
1b944a9b-fdae-4f97-aeb1-b7eea0beac53


In [3]:
# choose a name for experiment
experiment_name = 'ml-experiment-1'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)

experiment



Name,Workspace,Report Page,Docs Page
ml-experiment-1,quick-starts-ws-132429,Link to Azure Machine Learning studio,Link to Documentation


In [5]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster

amlcompute_cluster_name = "notebook132429"

# Verify that cluster does not exist already

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# For a more detailed view of current AmlCompute status, use get_status().


Found existing cluster, use it.

Running


In [ ]:
# dataset = Dataset.get_by_name(ws, name='heart')
# ds = dataset.to_pandas_dataframe()
# ds

In [6]:
#upload Heart Failure Dataset

data = pd.read_csv("heart_failure_clinical_records_dataset.csv")
data.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [7]:
train, test = train_test_split(data, test_size=0.2, random_state=1 )

if "training" not in os.listdir():
    os.mkdir("./training")

if "models" not in os.listdir():
    os.mkdir("./models")

train.to_csv("training/train_data.csv", index = False)

data_store = ws.get_default_datastore()
data_store.upload(src_dir = "./training", target_path = 'udacity-project', overwrite = True,  show_progress = True)


Uploading an estimated of 1 files
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_8e99f9b1b8744b0f887a40d672893118

In [8]:
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("udacity-project/train_data.csv"))
# train=TabularDatasetFactory.from_delimited_files([(data_store, 'training/train_data.csv')])


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

We are trying to predict DEATH_EVENT, therefore we need to define the task as classification. Also define Featurization as auto and enable early stopping and the primary metric AUC_weighted

In [9]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'    
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    
    compute_target = compute_target,
    task = "classification",
    training_data = train_data,
    label_column_name = "DEATH_EVENT",   
    path = project_folder,
    enable_early_stopping= True,
    featurization= 'auto',
    debug_log = "automl_errors.log",
    **automl_settings 
)

In [ ]:
# TODO: Submit your experiment

In [10]:
remote_run = experiment.submit(automl_config, show_output = True)
RunDetails(remote_run).show()


Running on remote.
No run_configuration provided, running on notebook132429 with default configuration
Running on remote compute: notebook132429
Parent Run ID: AutoML_b5e41e6c-d318-45f0-9c08-c3653e9d943d

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTIO

{'runId': 'AutoML_b5e41e6c-d318-45f0-9c08-c3653e9d943d',
 'target': 'notebook132429',
 'status': 'Completed',
 'startTimeUtc': '2020-12-30T02:56:01.192127Z',
 'endTimeUtc': '2020-12-30T03:15:40.112599Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'notebook132429',
  'AMLSettingsJsonString': '{"path":null,"name":"ml-experiment-1","subscription_id":"1b944a9b-fdae-4f97-aeb1-b7eea0beac53","resource_group":"aml-quickstarts-132429","workspace_name":"quick-starts-ws-132429","region":"southcentralus","compute_target":"notebook132429","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":nul

In [ ]:
#Take SS

RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_model, fitted_model = remote_run.get_output()

print(best_model)

# Take SS

Run(Experiment: ml-experiment-1,
Id: AutoML_b5e41e6c-d318-45f0-9c08-c3653e9d943d_43,
Type: azureml.scriptrun,
Status: Completed)


In [13]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.056842105263157895,
                                                                                                    min_weight_fraction_leaf=0.0,
                                                                                                    n_estimators=600,
    

In [14]:
#TODO: Save the best model
joblib.dump(fitted_model, "models/automl_hearth.pkl")


['models/automl_hearth.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [23]:
model = Model.register(model_path = "./models/automl_hearth.pkl",
                      model_name = "automl_hearth.pkl",
                      description = "Best model trained with AutoML",
                      workspace = ws, model_framework=Model.Framework.SCIKITLEARN)

# model = remote_run.register_model(model_name='automl_hearth.pkl', 
#                                  model_path='./models/automl_hearth.pkl',
#                                  model_framework=Model.Framework.SCIKITLEARN,
#                                  workspace = ws)

Registering model automl_hearth.pkl


In [21]:
# Define Deployment

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2,
                                              enable_app_insights=True, auth_enabled=True) 

# deployment_config = AciWebservice.deploy_configuration()


In [ ]:
# Create the environment
# myenv = Environment(workspace=ws, name="myenv")
# 
# inference_config = InferenceConfig(entry_script="score.py",
#                                    environment=myenv)

In [24]:
service_name = 'my-sklearn-service'

service = Model.deploy(ws, service_name, [model], deployment_config=aci_config)

# service = Model.deploy(ws, 'myservice2', [model], inference_config, deployment_config)
# service.update(enable_app_insights=True)

In [25]:
service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.........................................................................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: cc46194d-c3c6-48ce-ae6f-70818c15cd01
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: my-sklearn-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image 896ee77023f74bc78f17c761d25ae40f.azurecr.io/azureml/azureml_9febe3c381178587c812a6e9134e214b locally. Please refer to https://aka.ms/debugimage#se

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: cc46194d-c3c6-48ce-ae6f-70818c15cd01
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: my-sklearn-service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.\nYou can also try to run image 896ee77023f74bc78f17c761d25ae40f.azurecr.io/azureml/azureml_9febe3c381178587c812a6e9134e214b locally. Please refer to https://aka.ms/de

In [ ]:
print("scoring URI: " + service2.scoring_uri)


TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
scoring_uri = 'scoring uri for your service'
headers = {'Content-Type':'application/json'}

test_data = json.dumps({'data':[{
    'age':75,
    'anaemia':0,
    'creatinine_phosphokinase':582,
    'diabetes':0,
    'ejection_fraction':20,
    'high_blood_pressure':1,
    'platelets':265000,
    'serum_creatinine':1.9,
    'serum_sodium':130,
    'sex':1,
    'smoking':0,
    'time':4}
    ]
        })

response = requests.post(scoring_uri, data=test_data, headers=headers)

print("Result:",response.text)

In [ ]:
# myenv.save_to_directory('./training',overwrite=False)

print(service.get_logs())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(response.status_code)
print(response.elapsed)
print(response.json())
service.delete()
